In [1]:
from proj_utils.fake_input_generator import generator
from proj_utils.input_interpreter import format_input

import matplotlib.pyplot as plt

from VRP.greedy_solutions import nearest_neighbour_solution
from VRP.modify_solution import combined_rand_modification
from VRP.verify_solution import valid_path, calculate_path_distance

from proj_utils.path import print_path

from solver_step import SolverStep

import simulated_annealing.solver as SA
import taboo_search.solver as TS

from multi_agent.VRP_model import VRPSolutionModel
%matplotlib widget

In [2]:
truck_volume_capacity = 15
truck_weight_capacity = 10
truck_capacity = (truck_volume_capacity, truck_weight_capacity)
dists, demand, points = generator(40)

In [3]:
eval_function = lambda x: calculate_path_distance(x, dists) if(valid_path(x,truck_capacity,demand)) else float('inf')

In [4]:
solver_step_SA = SA.generic_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    )

In [5]:
taboo_list_size = 10
solver_step_TS = TS.taboo_search_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    taboo_list_size
    )

In [8]:
init_state = nearest_neighbour_solution(dists, demand, truck_capacity)
init_step = SolverStep(0,init_state,eval_function(init_state),5)
step_function_list = [solver_step_SA, solver_step_TS]

In [9]:
sol_model = VRPSolutionModel(init_step, step_function_list)

In [10]:
for _ in range(100):
    sol_model.step()

In [11]:
sol_states = sol_model.datacollector.get_model_vars_dataframe()
print(sol_states)

                                        TheGlobalBest  TheGlobalBestValue
0   [0, 1, 26, 38, 18, 31, 35, 29, 20, 19, 36, 0, ...          928.501486
1   [0, 1, 26, 38, 18, 31, 35, 29, 20, 19, 36, 0, ...          928.501486
2   [0, 1, 26, 38, 18, 31, 35, 29, 20, 19, 36, 0, ...          916.762859
3   [0, 1, 26, 38, 18, 11, 35, 29, 20, 19, 36, 0, ...          917.005724
4   [0, 1, 26, 38, 18, 11, 35, 29, 20, 19, 36, 0, ...          917.005724
..                                                ...                 ...
95  [0, 26, 38, 31, 19, 20, 29, 35, 36, 11, 1, 0, ...          946.335990
96  [0, 26, 38, 31, 19, 20, 29, 35, 36, 11, 1, 0, ...          946.335990
97  [0, 26, 38, 31, 19, 20, 29, 35, 36, 11, 1, 0, ...          946.335990
98  [0, 26, 38, 31, 19, 20, 29, 35, 36, 11, 1, 0, ...          946.335990
99  [0, 26, 38, 31, 19, 20, 29, 35, 36, 11, 1, 0, ...          946.433927

[100 rows x 2 columns]
